In [ ]:
import numpy as np
import pandas as pd
import torch
from scipy.sparse import save_npz, load_npz
import os
import json

In [ ]:
import melloddy_tuner
from melloddy_tuner.utils.single_row_prep2pred import SingleRowPreparator, KeyProviderTrivial, KeyProviderFromEnv
from melloddy_predictor.predictor_single import PredictorSingle
from melloddy_tuner.tunercli import do_prepare_prediction_online
from melloddy_tuner.utils.standardizer import Standardizer
from melloddy_tuner.utils.descriptor_calculator import DescriptorCalculator

import sparsechem

Specify environment and paths

In [ ]:
#conda environment to use when running shell script for reference output
conda_env = " melloddy_peipeline_dev"

root_path = os.path.dirname(os.getcwd())
test_path = os.path.join(root_path,"tests/begin_to_end_test")
model_path = os.path.join(root_path,"inputs/models")
config_path = os.path.join(root_path,"inputs/config")

params_file = os.path.join(config_path,"example_parameters.json")
key_file = os.path.join(config_path,"example_key.json")

tunercli_path = os.path.dirname(melloddy_tuner.__file__)
predict_py_path = os.path.dirname(sparsechem.__file__)

In [ ]:
os.environ["MELLODDY_KEY"] = "melloddy" #trivial key for testing purposes
key_provider = KeyProviderFromEnv("MELLODDY_KEY")

read input data

In [ ]:
smiles_df = pd.read_csv(os.path.join(test_path,"T2_100samples.csv"))
smiles_df.head()

pepare the x-matrix with conventional functions

In [ ]:
x_matrix, df_failed, mapping_table = do_prepare_prediction_online(input_structure = smiles_df, key_path = key_file, config_file = params_file, num_cpu = 1)

mt_out_folder = os.path.join(test_path, "mt_output")
os.makedirs(mt_out_folder,exist_ok=True)
save_npz(os.path.join(mt_out_folder,"pred_x.npz"), x_matrix)
mapping_table.to_csv(os.path.join(mt_out_folder,"mapping_table.csv"),index=False)

create script to generate reference output

In [ ]:
script = """
conda activate {cenv}

cd {sp_path}
mkdir -p {test_path}/sc_output
python predict.py --x {test_path}/mt_output/pred_x.npz --dev cpu --conf {model_path}/example_cls_model/hyperparameters.json --model {model_path}/example_cls_model/model.pth --outprefix {test_path}/sc_output/cls_model
python predict.py --x {test_path}/mt_output/pred_x.npz --dev cpu --conf {model_path}/example_clsaux_model/hyperparameters.json --model {model_path}/example_clsaux_model/model.pth --outprefix {test_path}/sc_output/clsaux_model
python predict.py --x {test_path}/mt_output/pred_x.npz --dev cpu --conf {model_path}/example_reg_model/hyperparameters.json --model {model_path}/example_reg_model/model.pth --outprefix {test_path}/sc_output/reg_model --inverse_normalization 1
python predict.py --x {test_path}/mt_output/pred_x.npz --dev cpu --conf {model_path}/example_hyb_model/hyperparameters.json --model {model_path}/example_hyb_model/model.pth --outprefix {test_path}/sc_output/hyb_model --inverse_normalization 1
"""
ref_gen_sh = script.format(tcpath=tunercli_path,test_path=test_path,pfile=params_file,kfile=key_file,sp_path=predict_py_path,model_path=model_path,cenv=conda_env)
ref_gen_sh

In [ ]:
script_file_name = os.path.join(test_path,"make_reference_output.sh")
with open(script_file_name ,"w") as script_file:
    script_file.write(ref_gen_sh)

**You need now to generate you reference outpt as egnerated through the command line tools** . Launch the command as generated by the output below in a shell

In [ ]:
print("source "+script_file_name)

Read the reference input

generate X-input from smiles

In [ ]:
my_srprep = SingleRowPreparator(params = params_file, key_provider = key_provider)

In [ ]:
x_tensors_test = {k:my_srprep.process_smiles(smi) for k,smi in smiles_df.set_index("input_compound_id")["smiles"].items()}

In [ ]:
x_dataframe_test = pd.concat({ix:pd.Series(v.to_dense().numpy()[0]) for ix,v in x_tensors_test.items()}).unstack().astype("int8")

In [ ]:
x_dataframe_test

load the reference x_csr matrix

In [ ]:
x_csr_ref = load_npz(os.path.join(test_path,"mt_output/pred_x.npz"))

In [ ]:
x_ndarray_ref = x_csr_ref.toarray()

In [ ]:
x_dataframe_ref = pd.DataFrame(x_ndarray_ref).rename(index=mapping_table.set_index("cont_descriptor_vector_id")["input_compound_id"])
x_dataframe_ref

compare x-matrices

In [ ]:
x_mat_test = np.equal(x_dataframe_test.sort_index().values, x_dataframe_ref.sort_index().values).all()
x_mat_test

### Test the predictions

Define some task slices for testing

In [ ]:
class_task_map = {'class_570':570,'class_581':581,'class_2276':2276}
regr_task_map = {"regr_633":633,"regr_740":740,"regr_2":2}

In [ ]:
my_preds = {'cls' : PredictorSingle(model= os.path.join(model_path,"example_cls_model/model.pth"), conf=os.path.join(model_path,"example_cls_model/hyperparameters.json"), class_task_map = class_task_map),\
           'clsaux' : PredictorSingle(model= os.path.join(model_path,"example_clsaux_model/model.pth"), conf=os.path.join(model_path,"example_clsaux_model/hyperparameters.json"), class_task_map = class_task_map),\
           'reg' : PredictorSingle(model= os.path.join(model_path,"example_reg_model/model.pth"), conf=os.path.join(model_path,"example_reg_model/hyperparameters.json"), regr_task_map = regr_task_map),\
           'hyb' : PredictorSingle(model= os.path.join(model_path,"example_hyb_model/model.pth"), conf=os.path.join(model_path,"example_hyb_model/hyperparameters.json"), class_task_map = class_task_map, regr_task_map = regr_task_map)
          }

In [ ]:
cls_res = {}
reg_res = {}
for mtype, my_pred in my_preds.items():
    for ix, x_tens in x_tensors_test.items():
        y_class_array, y_regr_array = my_pred.predict_from_tensor(x_tens)
        if y_class_array.shape[1] > 0:
            if not mtype in cls_res:
                cls_res[mtype] = {}
            cls_res[mtype][ix] = y_class_array
        if y_regr_array.shape[1] > 0:
            if not mtype in reg_res:
                reg_res[mtype] = {}
            reg_res[mtype][ix] = y_regr_array
    

In [ ]:
cls_res.keys()

In [ ]:
cls_res_df_test = {mtype : pd.concat({i: pd.Series(j[0]) for i,j in cls_res[mtype].items()}).unstack() for mtype in cls_res.keys()}
reg_res_df_test = {mtype : pd.concat({i: pd.Series(j[0]) for i,j in reg_res[mtype].items()}).unstack() for mtype in reg_res.keys()}

Read in reference prediction output

In [ ]:
y_refs_class = {"cls":np.load(os.path.join(test_path,"sc_output/cls_model-class.npy")),\
                "clsaux": np.load(os.path.join(test_path,"sc_output/clsaux_model-class.npy")),\
                "hyb": np.load(os.path.join(test_path,"sc_output/hyb_model-class.npy"))}
y_refs_regr = {"reg": np.load(os.path.join(test_path,"sc_output/reg_model-regr.npy")),\
               "hyb": np.load(os.path.join(test_path,"sc_output/hyb_model-regr.npy"))}

In [ ]:
y_refs_class_dfs = {mtype : pd.DataFrame(y).rename(index=mapping_table.set_index("cont_descriptor_vector_id")["input_compound_id"]) for mtype, y in y_refs_class.items()}
y_refs_regr_dfs = {mtype : pd.DataFrame(y).rename(index=mapping_table.set_index("cont_descriptor_vector_id")["input_compound_id"]) for mtype, y in y_refs_regr.items()}

In [ ]:
res_class = {mtype : np.allclose(y_df_test.sort_index().values,y_refs_class_dfs[mtype].sort_index().values) for mtype, y_df_test in cls_res_df_test.items()}
res_class

We reproduce the classification predictions exactly, as values above a true

In [ ]:
res_regr = {mtype : np.allclose(y_df_test.sort_index().values,y_refs_regr_dfs[mtype].sort_index().values) for mtype, y_df_test in reg_res_df_test.items()}
res_regr

We fail however to reproduce regression results, as the values above are false. This could be due to sparsechem issue https://github.com/melloddy/SparseChem/issues/5, meaning that the dense predictions from predict.py are not inversely normnalized. Let's try to normalize those reference predictions ourselves and then retry

In [ ]:
y_refs_regr_inorm_dfs = {mtype : y*my_preds[mtype].reg_stddev + my_preds[mtype].reg_mean for mtype,y in y_refs_regr_dfs.items()}

In [ ]:
y_refs_regr_inorm = {mtype : y*my_preds[mtype].reg_stddev + my_preds[mtype].reg_mean for mtype,y in y_refs_regr.items()}

In [ ]:
res_regr_inorm = {mtype : np.allclose(y_df_test.sort_index().values,y_refs_regr_inorm_dfs[mtype].sort_index().values) for mtype, y_df_test in reg_res_df_test.items()}
res_regr_inorm

Still not working!!

In [ ]:
colmask = ((reg_res_df_test['reg'] > 10) | (reg_res_df_test['reg'] < 3)).sum(axis=0) == 0

In [ ]:
res_regr_inorm = {mtype : np.allclose(y_df_test.sort_index().values[:,colmask],y_refs_regr_inorm_dfs[mtype].sort_index().values[:,colmask]) for mtype, y_df_test in reg_res_df_test.items()}
res_regr_inorm

### Test single row predictor object

In [ ]:
%%time
y_res_slice = {}
for k,smi in smiles_df.set_index("input_compound_id")["smiles"].items():
    try: 
        x = my_srprep.process_smiles(smi)
        y = my_preds["hyb"].predict_decorated_series_from_tensor(x)
        y_res_slice[k] = y
    except Error as e:
        print("prediction for {0} raised and error {1}".format(k, e.str()))


In [ ]:
res_slice_df = pd.concat(y_res_slice).unstack()
res_slice_df.index.names = ['input_compound_id']
res_slice_df.to_csv(os.path.join(test_path,"selected_hyb_tasks.csv"),index=True)


In [ ]:
res_slice_df

In [ ]:
pd.read_csv(os.path.join(test_path,"selected_hyb_tasks.csv"),index_col = "input_compound_id")

In [ ]:
smiles_failing_df = pd.read_csv(os.path.join(test_path,"T2_100samples_failing.csv"))
smiles_failing_df.head()

In [ ]:
%%time
y_failing_res_slice = {}
for k,smi in smiles_failing_df.set_index("input_compound_id")["smiles"].items():
    try: 
        x = my_srprep.process_smiles(smi)
        y = my_preds["hyb"].predict_decorated_series_from_tensor(x)
        y_failing_res_slice[k] = y
    except Exception as e:
        print("prediction for {0} raised an {1}: {2}".format(k, type(e), e))


In [ ]:
pd.concat(y_failing_res_slice).unstack()

In [ ]:
my_srprep_trusting = SingleRowPreparator(key_provider = key_provider, params = params_file, trust_standardization = False)

In [ ]:
%%time
y_trusting_res_slice = {}
for k,smi in smiles_df.set_index("input_compound_id")["smiles"].items():
    try: 
        x = my_srprep_trusting.process_smiles(smi)
        y = my_preds["hyb"].predict_decorated_series_from_tensor(x)
        y_trusting_res_slice[k] = y
    except Exception as e:
        print("prediction for {0} raised and error {1}".format(k, e))


In [ ]:
pd.concat(y_trusting_res_slice).unstack() - pd.concat(y_res_slice).unstack()

In [ ]:
pd.read_csv(os.path.join(mt_out_folder,"mapping_table.csv"))

In [ ]:
y_refs_selected_class_tasks = y_refs_class["hyb"][:,np.array(list(class_task_map.values()))]
y_refs_selected_regr_tasks = y_refs_regr_inorm["hyb"][:,np.array(list(regr_task_map.values()))]

In [ ]:
y_refs_select_class_df = pd.DataFrame(y_refs_selected_class_tasks, columns = list(class_task_map.keys())).rename(index=mapping_table.set_index("cont_descriptor_vector_id")["input_compound_id"])
y_refs_select_class_df 

In [ ]:
y_refs_select_regr_df = pd.DataFrame(y_refs_selected_regr_tasks, columns = list(regr_task_map.keys())).rename(index=mapping_table.set_index("cont_descriptor_vector_id")["input_compound_id"])
y_refs_select_regr_df 

In [ ]:
res_slice_ref_df = pd.concat([y_refs_select_class_df, y_refs_select_regr_df],axis=1)
res_slice_ref_df.sort_index()

In [ ]:
from pandas._testing import assert_frame_equal

In [ ]:
res_slice_ref_df.index.names = ['input_compound_id']
assert_frame_equal(res_slice_df.sort_index().astype("float32"), res_slice_ref_df.sort_index().astype("float32"))